In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.layers import Input , Dense , Reshape , Flatten
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.models import Sequential , Model
from keras.optimizers import Adam
from keras.datasets import mnist
from tqdm.notebook import tqdm

In [2]:
img_rows=28
img_cols=28
channels=1
img_shape=(img_rows,img_cols,channels)

In [3]:
def build_generator():
    noise_shape=(100,)
    model=Sequential()
    
    model.add(Dense(256,input_shape=noise_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    model.add(Dense(np.prod(img_shape),activation='tanh'))
    model.add(Reshape(img_shape))
    
    model.summary()
    
    noise=Input(shape=noise_shape)
    
    img=model(noise)
    
    return Model(noise, img)

In [4]:
np.prod(img_shape)

784

In [5]:
def build_discriminator():
    model=Sequential()
    
    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Dense(1,activation='sigmoid'))
    
    model.summary()
    
    img=Input(shape=img_shape)
    validity=model(img)
    
    return Model(img, validity)


In [6]:
def train(X_train,epochs,batch_size=128,save_interval=100):
    
    X_train=(X_train.astype(np.float32)-127.5)/127.5
    X_train=np.expand_dims(X_train,axis=3)
    
    half_batch=int(batch_size/2)
    
    for epoch in range(epochs):
        #Train Discriminator
        
        idx=np.random.randint(0,X_train.shape[0],half_batch)
        imgs=X_train[idx]
        
        noise=np.random.normal(0,1,(half_batch,100))
        
        # Generate a half batch of fake images
        gen_imgs=generator.predict(noise)
        
        d_loss_real=discriminator.train_on_batch(imgs,np.ones((half_batch,1)))
        d_loss_fake=discriminator.train_on_batch(gen_imgs,np.zeros((half_batch,1)))
        d_loss=0.5*np.add(d_loss_real,d_loss_fake)
        
        #Train Generator
        noise=np.random.normal(0,1,(batch_size,100))
        valid_y=np.array([1]*batch_size)
        g_loss=combined.train_on_batch(noise,valid_y)
        print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
        
        if epoch % save_interval==0:
            save_imgs(epoch)
            

In [7]:
def save_imgs(epoch):
    r,c=5,5
    noise=np.random.normal(0,1,(r*c,100))
    gen_imgs=generator.predict(noise)
    
    #Rescale images 0-1
    gen_imgs=0.5*gen_imgs + 0.5
    fig, axs=plt.subplots(r,c)
    cnt=0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt,:,:,0],cmap='gray')
            axs[i,j].axis('off')
            cnt+=1
    fig.savefig("images//mnist_%d.png" % epoch)
    plt.close()
    

In [8]:
optimizer=Adam(0.0002,0.5) #Learning rate and momentum

In [9]:
discriminator=build_discriminator()
discriminator.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])

C:\Users\iitg\anaconda3\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\iitg\anaconda3\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │         401,920 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu (LeakyReLU)              │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_1 (LeakyReLU)            │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 533,505 (2.04 MB)

 Trainable params: 533,505 (2.04 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
generator=build_generator()
generator.compile(loss='binary_crossentropy',optimizer=optimizer)


C:\Users\iitg\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                      │ (None, 256)                 │          25,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_2 (LeakyReLU)            │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_3 (LeakyReLU)            │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1024)                │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_4 (LeakyReLU)            │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 1024)                │           4,096 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 784)                 │         803,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 28, 28, 1)           │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,493,520 (5.70 MB)

 Trainable params: 1,489,936 (5.68 MB)

 Non-trainable params: 3,584 (14.00 KB)

In [11]:
z=Input(shape=(100,))
img=generator(z)

In [12]:
img

<KerasTensor shape=(None, 28, 28, 1), dtype=float32, sparse=False, name=keras_tensor_94>

In [13]:
discriminator.trainable=False

In [14]:
valid=discriminator(img)

In [15]:
valid

<KerasTensor shape=(None, 1), dtype=float32, sparse=False, name=keras_tensor_95>

In [16]:
combined=Model(z,valid)
combined.compile(loss='binary_crossentropy',optimizer=optimizer)

In [17]:
(X_train,_),(_,_)=mnist.load_data()

In [ ]:
tqdm(train(X_train,epochs=10001,batch_size=32,save_interval=200))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 520ms/step


C:\Users\iitg\anaconda3\Lib\site-packages\keras\src\backend\tensorflow\trainer.py:74: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


0 [D loss: 0.851646, acc.: 17.19%] [G loss: 0.616477]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1 [D loss: 0.825358, acc.: 20.83%] [G loss: 0.615341]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
2 [D loss: 0.812623, acc.: 22.19%] [G loss: 0.604092]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
3 [D loss: 0.808450, acc.: 22.49%] [G loss: 0.594801]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
4 [D loss: 0.808582, acc.: 23.37%] [G loss: 0.578706]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
5 [D loss: 0.813140, acc.: 23.37%] [G loss: 0.567888]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
6 [D loss: 0.818410, acc.: 22.72%] [G loss: 0.557077]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
7 [D loss: 0.823542, acc.: 21.80%] [G loss: 0.546901]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
8 [D loss: 0.835080, acc.: 20.02%] [G loss: 0.534178]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
9 [D loss: 0.844621, acc.: 19.88%] [G loss: 0.525215]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
10 [D loss: 0.856694, acc.: 19.05%] [G lo

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
53 [D loss: 1.233853, acc.: 14.88%] [G loss: 0.270474]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
54 [D loss: 1.242510, acc.: 14.84%] [G loss: 0.267474]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
55 [D loss: 1.250013, acc.: 14.80%] [G loss: 0.264492]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
56 [D loss: 1.258412, acc.: 14.65%] [G loss: 0.261386]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
57 [D loss: 1.266619, acc.: 14.50%] [G loss: 0.258694]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
58 [D loss: 1.272924, acc.: 14.42%] [G loss: 0.255842]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
59 [D loss: 1.279548, acc.: 14.54%] [G loss: 0.253229]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
60 [D loss: 1.286911, acc.: 14.51%] [G loss: 0.250423]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
61 [D loss: 1.293862, acc.: 14.47%] [G loss: 0.247988]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
62 [D loss: 1.301154, acc.: 14.44%] [G loss: 0.245336]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
63 [D loss: 1.309132, acc.: 14.3

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
117 [D loss: 1.628335, acc.: 13.69%] [G loss: 0.154783]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
118 [D loss: 1.633838, acc.: 13.63%] [G loss: 0.153809]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
119 [D loss: 1.638612, acc.: 13.62%] [G loss: 0.152786]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
120 [D loss: 1.642716, acc.: 13.64%] [G loss: 0.151808]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
121 [D loss: 1.646636, acc.: 13.66%] [G loss: 0.150887]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
122 [D loss: 1.651522, acc.: 13.67%] [G loss: 0.149893]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
123 [D loss: 1.656897, acc.: 13.66%] [G loss: 0.148873]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
124 [D loss: 1.661252, acc.: 13.65%] [G loss: 0.147989]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
125 [D loss: 1.665571, acc.: 13.67%] [G loss: 0.147011]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
126 [D loss: 1.670454, acc.: 13.73%] [G loss: 0.146073]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
127 [D loss: 1.674363,

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2520 [D loss: 3.789926, acc.: 13.32%] [G loss: 0.009566]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2521 [D loss: 3.790195, acc.: 13.32%] [G loss: 0.009562]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2522 [D loss: 3.790508, acc.: 13.32%] [G loss: 0.009559]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2523 [D loss: 3.790829, acc.: 13.33%] [G loss: 0.009555]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2524 [D loss: 3.791116, acc.: 13.33%] [G loss: 0.009551]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2525 [D loss: 3.791439, acc.: 13.33%] [G loss: 0.009548]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2526 [D loss: 3.791748, acc.: 13.32%] [G loss: 0.009544]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
2527 [D loss: 3.792055, acc.: 13.32%] [G loss: 0.009540]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2528 [D loss: 3.792385, acc.: 13.33%] [G loss: 0.009537]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2529 [D loss: 3.792722, acc.: 13.32%] [G loss: 0.009533]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2530 [D loss

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2583 [D loss: 3.808895, acc.: 13.33%] [G loss: 0.009339]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2584 [D loss: 3.809226, acc.: 13.33%] [G loss: 0.009336]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
2585 [D loss: 3.809514, acc.: 13.32%] [G loss: 0.009332]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2586 [D loss: 3.809796, acc.: 13.33%] [G loss: 0.009329]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2587 [D loss: 3.810121, acc.: 13.33%] [G loss: 0.009325]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2588 [D loss: 3.810440, acc.: 13.32%] [G loss: 0.009322]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
2589 [D loss: 3.810728, acc.: 13.32%] [G loss: 0.009318]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
2590 [D loss: 3.811062, acc.: 13.32%] [G loss: 0.009315]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2591 [D loss: 3.811355, acc.: 13.32%] [G loss: 0.009311]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2592 [D loss: 3.811636, acc.: 13.32%] [G loss: 0.009308]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2593 [D loss

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2646 [D loss: 3.827724, acc.: 13.35%] [G loss: 0.009123]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2647 [D loss: 3.827962, acc.: 13.35%] [G loss: 0.009120]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
2648 [D loss: 3.828228, acc.: 13.35%] [G loss: 0.009116]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2649 [D loss: 3.828507, acc.: 13.36%] [G loss: 0.009113]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2650 [D loss: 3.828798, acc.: 13.36%] [G loss: 0.009110]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
2651 [D loss: 3.829094, acc.: 13.36%] [G loss: 0.009106]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2652 [D loss: 3.829370, acc.: 13.36%] [G loss: 0.009103]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2653 [D loss: 3.829628, acc.: 13.36%] [G loss: 0.009100]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2654 [D loss: 3.829910, acc.: 13.36%] [G loss: 0.009096]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2655 [D loss: 3.830214, acc.: 13.36%] [G loss: 0.009093]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2656 [D loss

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2709 [D loss: 3.846083, acc.: 13.36%] [G loss: 0.008917]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2710 [D loss: 3.846342, acc.: 13.37%] [G loss: 0.008914]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2711 [D loss: 3.846622, acc.: 13.36%] [G loss: 0.008911]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2712 [D loss: 3.846920, acc.: 13.36%] [G loss: 0.008907]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2713 [D loss: 3.847199, acc.: 13.36%] [G loss: 0.008904]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2714 [D loss: 3.847483, acc.: 13.36%] [G loss: 0.008901]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2715 [D loss: 3.847744, acc.: 13.36%] [G loss: 0.008898]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2716 [D loss: 3.848001, acc.: 13.36%] [G loss: 0.008895]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2717 [D loss: 3.848247, acc.: 13.36%] [G loss: 0.008891]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2718 [D loss: 3.848489, acc.: 13.36%] [G loss: 0.008888]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2719 [D loss

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2772 [D loss: 3.863533, acc.: 13.36%] [G loss: 0.008720]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
2773 [D loss: 3.863820, acc.: 13.36%] [G loss: 0.008717]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2774 [D loss: 3.864069, acc.: 13.37%] [G loss: 0.008714]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
2775 [D loss: 3.864371, acc.: 13.37%] [G loss: 0.008711]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2776 [D loss: 3.864657, acc.: 13.37%] [G loss: 0.008708]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
2777 [D loss: 3.864950, acc.: 13.37%] [G loss: 0.008705]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2778 [D loss: 3.865223, acc.: 13.37%] [G loss: 0.008702]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2779 [D loss: 3.865509, acc.: 13.37%] [G loss: 0.008699]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2780 [D loss: 3.865800, acc.: 13.37%] [G loss: 0.008696]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2781 [D loss: 3.866082, acc.: 13.37%] [G loss: 0.008693]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2782 [D loss

In [ ]:
generator.save("1st Generator model.h5")